<a href="https://colab.research.google.com/github/imkvncpr/MITXPro-DeepLearning/blob/main/Copy_of_CapstonePrjtKevonCooper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
Model = tf.keras.models.Model
Sequential = tf.keras.models.Sequential
Dense = tf.keras.layers.Dense
LSTM = tf.keras.layers.LSTM
Concatenate = tf.keras.layers.Concatenate
Input = tf.keras.layers.Input
Dropout = tf.keras.layers.Dropout
EarlyStopping = tf.keras.callbacks.EarlyStopping
ModelCheckpoint = tf.keras.callbacks.ModelCheckpoint
import plotly.graph_objects as go
import logging
from typing import Tuple, Dict, Optional, Union
import os

# Step 1: Set up logging configuration for debugging and monitoring
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class CryptoPricePredictor:
    def __init__(self,
                 ticker: str = 'BTC-USD',
                 look_back: int = 60,
                 test_size: float = 0.2,
                 random_state: int = 42):
        """
        Step 2: Initialize the predictor with configuration parameters
        - ticker: The cryptocurrency symbol to predict (e.g., BTC-USD)
        - look_back: Number of past days to consider for prediction
        - test_size: Portion of data reserved for testing
        - random_state: Seed for reproducibility
        """
        self.ticker = ticker
        self.look_back = look_back
        self.test_size = test_size
        self.random_state = random_state
        self.scaler = MinMaxScaler(feature_range=(0, 1))  # Scale data between 0 and 1
        self.model = None
        self.X = None
        self.last_sequence = None  # Used for future predictions

    def download_crypto_data(self, years: int = 5) -> Optional[pd.DataFrame]:
        """
        Step 3: Download historical price data using yfinance
        - Fetches daily OHLCV data for specified number of years
        - Returns None if download fails
        """
        try:
            end_date = pd.Timestamp.now()
            start_date = end_date - pd.DateOffset(years=years)
            data = yf.download(self.ticker, start=start_date, end=end_date)
            logger.info(f"Successfully downloaded {years} years of {self.ticker} data")
            return data
        except Exception as e:
            logger.error(f"Error downloading data: {str(e)}")
            return None

    def prepare_data(self, data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """
        Step 4: Prepare data for LSTM model
        - Creates sequences of look_back days for training
        - Scales all features using MinMaxScaler
        - Returns X (features) and y (target) arrays
        """
        try:
            # Extract relevant features from data
            feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
            data_processed = data[feature_columns].values

            # Scale features to range [0,1]
            scaled_data = self.scaler.fit_transform(data_processed)

            # Create sequences for LSTM
            X, y = [], []
            for i in range(self.look_back, len(scaled_data)):
                X.append(scaled_data[i-self.look_back:i])  # Sequence of look_back days
                y.append(scaled_data[i, 3])  # Target is next day's closing price

            X, y = np.array(X), np.array(y)
            self.last_sequence = X[-1:]  # Save last sequence for future predictions
            logger.info(f"Prepared data shapes: X: {X.shape}, y: {y.shape}")
            return X, y
        except Exception as e:
            logger.error(f"Error preparing data: {str(e)}")
            raise

    def create_model(self,
                    neurons: int = 100,
                    dropout: float = 0.3) -> tf.keras.models.Sequential:
        """
        Step 5: Create LSTM neural network architecture
        - Two LSTM layers with decreasing units
        - Dropout layers for regularization
        - Dense layers for final prediction
        """
        try:
            model = Sequential([
                # First LSTM layer with return sequences for stacking
                LSTM(neurons, return_sequences=True,
                     input_shape=(self.look_back, 5)),
                Dropout(dropout),  # Prevent overfitting

                # Second LSTM layer without return sequences
                LSTM(neurons//2, return_sequences=False),
                Dropout(dropout/2),

                # Dense layers for final prediction
                Dense(25, activation='relu'),
                Dense(1)  # Output layer for price prediction
            ])
            model.compile(optimizer='adam', loss='mean_squared_error')
            return model
        except Exception as e:
            logger.error(f"Error creating model: {str(e)}")
            raise

    def train_model(self,
                   X: np.ndarray,
                   y: np.ndarray,
                   optimize_hyperparameters: bool = False,
                   **kwargs) -> Tuple[tf.keras.models.Sequential, Dict[str, float]]:
        """
        Step 6: Train the LSTM model
        - Splits data into train/test sets
        - Optional hyperparameter optimization
        - Uses early stopping to prevent overfitting
        - Returns trained model and performance metrics
        """
        try:
            # Split data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=self.test_size, random_state=self.random_state
            )

            if optimize_hyperparameters:
                model = self._optimize_hyperparameters(X_train, y_train)
            else:
                model = self.create_model()

                # Set up callbacks for training
                callbacks = [
                    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
                    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
                ]

                # Train the model
                model.fit(
                    X_train, y_train,
                    epochs=kwargs.get('epochs', 100),
                    batch_size=kwargs.get('batch_size', 32),
                    validation_split=0.2,
                    callbacks=callbacks,
                    verbose=1
                )

            # Evaluate model performance
            predictions = model.predict(X_test, verbose=0)
            metrics = self._calculate_metrics(y_test, predictions)

            self.model = model
            return model, metrics

        except Exception as e:
            logger.error(f"Error training model: {str(e)}")
            raise

    def predict_future(self, days: int = 30) -> np.ndarray:
        """
        Step 7: Generate future price predictions
        - Uses the last known sequence to predict next day
        - Updates sequence with prediction for next prediction
        - Returns array of predicted prices in original scale
        """
        if self.model is None:
            raise ValueError("Model must be trained before making predictions")

        if self.last_sequence is None:
            raise ValueError("No data sequence available for prediction")

        try:
            current_sequence = np.copy(self.last_sequence)
            future_predictions = []

            # Predict prices for specified number of days
            for _ in range(days):
                # Predict next day's price
                next_pred = self.model.predict(current_sequence, verbose=0)[0]
                future_predictions.append(next_pred)

                # Update sequence for next prediction
                new_sequence = np.copy(current_sequence)
                new_sequence[0, :-1] = new_sequence[0, 1:]
                new_sequence[0, -1] = new_sequence[0, -2]
                new_sequence[0, -1, 3] = next_pred

                current_sequence = new_sequence

            # Convert predictions back to original price scale
            future_predictions = np.array(future_predictions).reshape(-1, 1)
            future_predictions_reshaped = np.hstack([
                np.zeros((len(future_predictions), 3)),
                future_predictions,
                np.zeros((len(future_predictions), 1))
            ])

            future_prices = self.scaler.inverse_transform(future_predictions_reshaped)[:, 3]

            logger.info(f"Generated {days} days of future predictions")
            return future_prices

        except Exception as e:
            logger.error(f"Error making future predictions: {str(e)}")
            raise

    def _optimize_hyperparameters(self, X_train: np.ndarray, y_train: np.ndarray) -> tf.keras.models.Sequential:
        """
        Step 8: Optimize model hyperparameters
        - Uses RandomizedSearchCV for parameter tuning
        - Tests different combinations of neurons, dropout, batch size, and epochs
        """
        model = KerasRegressor(model=self.create_model, verbose=0)

        param_dist = {
            'model__neurons': [50, 100, 150, 200],
            'model__dropout': [0.2, 0.3, 0.4, 0.5],
            'batch_size': [16, 32, 64],
            'epochs': [50, 100, 150]
        }

        random_search = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_dist,
            n_iter=10,
            cv=3,
            verbose=2
        )

        random_search.fit(X_train, y_train)
        logger.info(f"Best hyperparameters: {random_search.best_params_}")
        return random_search.best_estimator_.model_

    def _calculate_metrics(self, y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
        """
        Step 9: Calculate performance metrics
        - Converts predictions back to original scale
        - Calculates MSE, RMSE, MAE, and R2 score
        """
        # Convert back to original price scale
        y_true_orig = self.scaler.inverse_transform(np.column_stack([np.zeros((len(y_true), 3)),
                                                                    y_true.reshape(-1, 1),
                                                                    np.zeros((len(y_true), 1))]))[:, 3]
        y_pred_orig = self.scaler.inverse_transform(np.column_stack([np.zeros((len(y_pred), 3)),
                                                                    y_pred.reshape(-1, 1),
                                                                    np.zeros((len(y_pred), 1))]))[:, 3]

        return {
            'MSE': mean_squared_error(y_true_orig, y_pred_orig),
            'RMSE': np.sqrt(mean_squared_error(y_true_orig, y_pred_orig)),
            'MAE': mean_absolute_error(y_true_orig, y_pred_orig),
            'R2': r2_score(y_true_orig, y_pred_orig)
        }

    def visualize_predictions(self, y_true: np.ndarray, y_pred: np.ndarray) -> None:
        """
        Step 10: Visualize predictions vs actual values
        - Creates interactive plot using plotly
        - Shows actual and predicted prices on same graph
        """
        # Convert predictions to original scale
        y_true_orig = self.scaler.inverse_transform(np.column_stack([np.zeros((len(y_true), 3)),
                                                                    y_true.reshape(-1, 1),
                                                                    np.zeros((len(y_true), 1))]))[:, 3]
        y_pred_orig = self.scaler.inverse_transform(np.column_stack([np.zeros((len(y_pred), 3)),
                                                                    y_pred.reshape(-1, 1),
                                                                    np.zeros((len(y_pred), 1))]))[:, 3]

        # Create interactive plot
        fig = go.Figure()
        fig.add_trace(go.Scatter(y=y_true_orig, mode='lines', name='Actual Prices'))
        fig.add_trace(go.Scatter(y=y_pred_orig, mode='lines', name='Predicted Prices'))
        fig.update_layout(
            title=f'{self.ticker} Price Prediction',
            xaxis_title='Time',
            yaxis_title='Price',
            template='plotly_dark'
        )
        fig.show()

def main():
    """
    Step 11: Main execution function
    - Demonstrates complete prediction pipeline
    - Handles errors and displays results
    """
    # Initialize predictor with default settings
    predictor = CryptoPricePredictor(ticker='BTC-USD', look_back=60)

    try:
        # Execute prediction pipeline
        logger.info("Downloading cryptocurrency data...")
        data = predictor.download_crypto_data(years=5)
        if data is None:
            logger.error("Failed to download data")
            return

        logger.info("Preparing data for training...")
        X, y = predictor.prepare_data(data)
        predictor.X = X

        logger.info("Training the model...")
        model, metrics = predictor.train_model(
            X, y,
            optimize_hyperparameters=False,
            epochs=100,
            batch_size=32
        )

        # Display results
        print("\nModel Performance Metrics:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.2f}")

        logger.info("Generating historical predictions visualization...")
        y_pred = model.predict(X, verbose=0)
        predictor.visualize_predictions(y, y_pred)

        logger.info("Generating future price predictions...")
        future_prices = predictor.predict_future(days=30)
        print("\nPredicted prices for next 30 days:")
        for i, price in enumerate(future_prices, 1):
            print(f"Day {i}: ${price:,.2f}")

    except Exception as e:
        logger.error(f"An error occurred in main: {str(e)}")
        raise

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0311

36/36 ━━━━━━━━━━━━━━━━━━━━ 9s 94ms/step - loss: 0.0307 - val_loss: 0.0037
Epoch 2/100
35/36 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0036

36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - loss: 0.0036 - val_loss: 0.0015
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 0.0029

36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - loss: 0.0028 - val_loss: 0.0013
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0022

...

36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 105ms/step - loss: 7.3118e-04 - val_loss: 3.8914e-04
Epoch 57/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 77ms/step - loss: 6.8957e-04 - val_loss: 5.2386e-04
Epoch 58/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - loss: 6.4414e-04 - val_loss: 8.2922e-04
Epoch 59/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - loss: 6.7441e-04 - val_loss: 4.3641e-04
Epoch 60/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - loss: 5.7213e-04 - val_loss: 4.8085e-04
Epoch 61/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step - loss: 7.1413e-04 - val_loss: 6.4896e-04
Epoch 62/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - loss: 9.0914e-04 - val_loss: 4.0695e-04
Epoch 63/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 7.5781e-04

36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - loss: 7.5754e-04 - val_loss: 3.6198e-04
Epoch 64/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - loss: 7.3958e-04 - val_loss: 3.6273e-04
Epoch 65/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 79ms/step - loss: 6.8432e-04 - val_loss: 4.3013e-04
Epoch 66/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - loss: 7.7458e-04 - val_loss: 3.7999e-04
Epoch 67/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - loss: 7.2118e-04 - val_loss: 4.6978e-04
Epoch 68/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - loss: 7.1485e-04 - val_loss: 6.1567e-04
Epoch 69/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 8.2033e-04

36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - loss: 8.1905e-04 - val_loss: 3.5959e-04
Epoch 70/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - loss: 6.7297e-04 - val_loss: 4.5836e-04
Epoch 71/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - loss: 5.7279e-04 - val_loss: 3.6028e-04
Epoch 72/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - loss: 5.5973e-04 - val_loss: 3.9311e-04
Epoch 73/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - loss: 6.0777e-04 - val_loss: 3.8951e-04
Epoch 74/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 124ms/step - loss: 5.7916e-04 - val_loss: 7.1053e-04
Epoch 75/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - loss: 7.1005e-04 - val_loss: 3.8919e-04
Epoch 76/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - loss: 5.9888e-04 - val_loss: 3.7795e-04
Epoch 77/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 112ms/step - loss: 5.7729e-04 - val_loss: 3.9627e-04
Epoch 78/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 4.8979e-04 - val_loss: 4.1048e-04
Epoch 79/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - loss:

<ipython-input-2-0cf8482df36d>:203: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)




Predicted prices for next 30 days:
Day 1: $96,204.85
Day 2: $95,676.09
Day 3: $95,270.55
Day 4: $94,993.96
Day 5: $94,825.28
Day 6: $94,734.35
Day 7: $94,693.29
Day 8: $94,681.18
Day 9: $94,684.70
Day 10: $94,696.33
Day 11: $94,712.22
Day 12: $94,730.42
Day 13: $94,749.79
Day 14: $94,769.52
Day 15: $94,788.88
Day 16: $94,807.28
Day 17: $94,824.27
Day 18: $94,839.65
Day 19: $94,853.26
Day 20: $94,865.16
Day 21: $94,875.39
Day 22: $94,884.15
Day 23: $94,891.55
Day 24: $94,897.80
Day 25: $94,902.99
Day 26: $94,907.31
Day 27: $94,910.87
Day 28: $94,913.80
Day 29: $94,916.19
Day 30: $94,918.12
